In [1]:
pip install lxml delta-spark beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen 
from io import BytesIO
from zipfile import ZipFile
request = requests.get("https://www.marktstammdatenregister.de/MaStR/Datendownload")
html = request.content
soup = BeautifulSoup(html, 'html.parser')
zip_url = soup.find('a', {'title': 'Download'}).get('href')
print(zip_url)

https://download.marktstammdatenregister.de/Gesamtdatenexport_20230103_22.2_5540291263d64b9997af23032250e7e8.zip


In [3]:
filename = (zip_url.split("/")[-1] ).rsplit('.', maxsplit=1)[0]
dataset_path = f"/home/jovyan/{filename}"
# Zip url to read data
zip_data = urlopen(zip_url)

# Read zipped data to temp directory
zFile = ZipFile(BytesIO(zip_data.read()))

#Extract zip data
zFile.extractall(dataset_path)

IncompleteRead: IncompleteRead(135338076 bytes read, 1092148901 more expected)

In [ ]:
#filename = (zip_url.split("/")[-1] ).rsplit('.', maxsplit=1)[0]
#dataset_path = f"/home/jovyan/{filename}"
#print(dataset_path)

In [ ]:
%%sh
rm -rf -r /home/jovyan/spark-warehouse/marktstammdaten_anlageneegsolar
rm -rf -r /home/jovyan/spark-warehouse/marktstammdaten_anlageneegspeicher
rm -rf -r /home/jovyan/spark-warehouse/marktstammdaten_anlagenstromspeicher
rm -rf -r /home/jovyan/spark-warehouse/marktstammdaten_einheitenstromspeicher
rm -rf -r /home/jovyan/spark-warehouse/marktstammdaten_einheitensolar

In [ ]:
#for databricks
import pandas as pd
import xml.etree.ElementTree as ET
#from sqlalchemy import create_engine
import os
from os import listdir
from os.path import isfile, join
import re
import zipfile
import pyspark
from pyspark.sql import SparkSession


In [ ]:
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("data-gurus") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.6.1")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
#spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")

# #Create SparkSession
# spark = SparkSession.builder.master("local[1]").appName("data-gurus").getOrCreate()
# print(spark)

In [ ]:
print(spark)
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

In [ ]:
# give path of dataset and file name without the number
def files_mask(dataset_path, filename):
    '''
    
    :param dataset_path: dataset path of zip folder
    :param filename: file name without extension type
    :return: matching list with the same kind of files
    '''
    # get all files names in the datasetfolder

    zf = zipfile.ZipFile(dataset_path, "r")
    all_files = []
    for file in zf.namelist():
        if file.endswith(".xml"):
            #[5:] removes DATA/ string from the beginning
            all_files.append(file)

    #all_files = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]

    regex_pattern = ".*" + filename + ".*"

    match_list = []
    for i in range(0, len(all_files)):
        match = re.findall(regex_pattern, all_files[i])
        if not match:
            # if list is empty
            pass
        else:
            match_list.append(match[0])

    # print("Required Files in the Dataset: ", match_list)
    return match_list

In [ ]:
def xml_to_dataframe_converter(dataset_path, xml_file):
    '''

    :param dataset_path:zip folder path 
    :param xml_file: full file name with the extension
    :return: pandas dataframe
    '''

    zf = zipfile.ZipFile(dataset_path, "r")

    if xml_file in zf.namelist():
        # print("xml_file > ",xml_file)
        xml_file_open = zf.open(xml_file)
        xml_file = xml_file_open.read()
        # print("xml_file_open > ",xml_file_open)
        
        df = pd.read_xml(xml_file, parser="lxml", encoding= "utf-16")

    #print(df.info())
    return df

In [ ]:
import warnings
warnings.filterwarnings('ignore') 

#this is the main function
def xml_parser():
    
    # change dataset path
    #dataset_path = "G:\DATA GURUS WORK\AAAA\"
    dataset_path = "/home/jovyan/Gesamtdatenexport_20221229_22.zip"
   
    file_names = ['AnlagenEegSolar', 'AnlagenEegSpeicher', 'AnlagenStromSpeicher', 'EinheitenStromSpeicher', 'EinheitenSolar']
    #file_names = ['EinheitenStromSpeicher']

    for i in range(0, len(file_names)):
    
        file_name = file_names[i]
        
        spark.sql(f"drop table if exists default.marktstammdaten_{file_name.lower()}")
        #dbutils.fs.rm(f"file:/home/jovyan/work/spark-warehouse/marktstammdaten_{file_name.lower()}", true)

        print("Current File Type : ", file_name)

        xml_files_list = files_mask(dataset_path, file_name)
        print("XML FILES LIST: ", xml_files_list)

        #print(schemaDict[file_name.lower()])      #TODO: delete it
        
        for xml_file in xml_files_list:

            print(f"Processing : {xml_file}")
            data = xml_to_dataframe_converter(dataset_path,xml_file)
            
            #print("pd data frame :", data)
            #print("pd df info :", data.info())
            
            if (file_name == "EinheitenSolar"):
                data.Hausnummer = data.Hausnummer.astype(str)
                data.NameStromerzeugungseinheit = data.NameStromerzeugungseinheit.astype(str)
                data.Bundesland = data.Bundesland.astype(str)
                data.Postleitzahl  = data.Postleitzahl .astype(str)
                data.Gemeindeschluessel = data.Gemeindeschluessel.astype(str)
                data.Lage = data.Lage.astype(str)
                data.Einsatzverantwortlicher = data.Einsatzverantwortlicher.astype(str)
                data.Einspeisungsart = data.Einspeisungsart.astype(str)
                data.Adresszusatz = data.Adresszusatz.astype(str)
                                
            if (file_name == "EinheitenStromSpeicher"):
                data.Technologie = data.Technologie.astype(str)
                data.Postleitzahl = data.Postleitzahl.astype(str)                   
                data.Einspeisungsart = data.Einspeisungsart.astype(str)                                 
                
            try:

                #creating spark data frame   
                sdf = spark.createDataFrame(data)
                #sdf.printSchema()
            except Exception as e:
                print("Error while converting from pandas to spark :", e)
    
            #database table name
            table_name = file_name.lower()
            table_name = "marktstammdaten_" + table_name

            try:            
                sdf.write.format("delta").option("overwriteSchema", "true").option("mergeSchema", "true").option("path", f"file:/home/jovyan/spark-warehouse/marktstammdaten_{file_name.lower()}").mode("append").saveAsTable(f'{table_name}')

            except Exception as e:
                print("Exception occurred while writing to databricks :")
                print(e)
                
    print("Finished processing!!!!")

In [ ]:
xml_parser()

In [ ]:
marktstammdaten_anlageneegsolar_df = spark.sql(f"select * from marktstammdaten_anlageneegsolar")
print(marktstammdaten_anlageneegsolar_df.count())
print(marktstammdaten_anlageneegsolar_df.schema)
#marktstammdaten_anlageneegsolar_df.show()


In [ ]:
marktstammdaten_anlageneegspeicher_df = spark.sql(f"select * from marktstammdaten_anlageneegspeicher")
print(marktstammdaten_anlageneegspeicher_df.count())
print(marktstammdaten_anlageneegspeicher_df.schema)
#marktstammdaten_anlageneegspeicher_df.show()


In [ ]:
marktstammdaten_anlagenstromspeicher_df = spark.sql(f"select * from marktstammdaten_anlagenstromspeicher")
print(marktstammdaten_anlagenstromspeicher_df.count())
print(marktstammdaten_anlagenstromspeicher_df.schema)
#marktstammdaten_anlagenstromspeicher_df.show()


In [ ]:
marktstammdaten_einheitenstromspeicher_df = spark.sql(f"select * from marktstammdaten_einheitenstromspeicher")
print(marktstammdaten_einheitenstromspeicher_df.count())
print(marktstammdaten_einheitenstromspeicher_df.schema)
#marktstammdaten_einheitenstromspeicher_df.show()


In [ ]:
marktstammdaten_einheitensolar_df = spark.sql(f"select * from marktstammdaten_einheitensolar")
print(marktstammdaten_einheitensolar_df.count())
print(marktstammdaten_einheitensolar_df.schema)
#marktstammdaten_einheitensolar_df.show()

%md
#### Finished processing
